In [1]:
#from load_data import load_data
#from other_predictors import shortest_path, common_neighbors #, hitting_time
#from predict_links import get_loc, delete_duplicates, predict
#from effective_transition import brute_effective_transition, weighted_effective_transition
#from spectral_embedding import spectral_embed
from scipy.sparse import linalg as sla
import scipy.sparse as ss

import networkx as nx
import numpy as np

import numpy.linalg as la
import time
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix

from functools import partial

from math import floor, ceil
from sklearn.neighbors import KDTree, BallTree
from scipy.spatial import cKDTree
from scipy.spatial.distance import pdist, squareform
from collections import Counter

In [22]:
def load_data(filename):
    myfile = open(filename, 'r')
    lines = myfile.readlines()
    lines = [line.split() for line in lines]
    time = 3 if len(lines[0])==4 else 2
    lines = [[int(line[0]),int(line[1]),int(line[time])] for line in lines]
    lines.sort(key=lambda x: x[2])
    n = len(lines)
    chop = floor(3*n/4)
    train = lines[:chop]
    to_train = [(edge[0],edge[1]) for edge in train]
    test = lines[chop:]
    k = len(test)
    to_predict = [(edge[0],edge[1]) for edge in test]
    G = nx.Graph()
    G.add_edges_from(to_train)
    G = max(nx.connected_component_subgraphs(G), key=len)
    lines = [[int(line[0]),int(line[1]),int(line[time])] for line in lines]
    lines.sort(key=lambda x: x[2])
    n = len(lines)
    chop = floor(3*n/4)
    train = lines[:chop]
    to_train = [(edge[0],edge[1]) for edge in train]
    G = nx.Graph()
    G.add_edges_from(to_train)
    G = max(nx.connected_component_subgraphs(G), key=len)
    nodes = G.nodes()
    test = lines[chop:]
    to_predict = [(edge[0],edge[1]) for edge in test if edge[0] in nodes and edge[1] in nodes]

    return G, to_predict

In [3]:
def all_closest(points, k):
    tree = KDTree(points)
    dists,inds = tree.query(points, k+1)
    n = points.shape[0]
    a = np.arange(n).reshape((n,1))
    mask = (inds != a)
    ret_dists = np.empty((n,k))
    ret_inds = np.empty((n,k))
    for i in xrange(n):
        mask = inds[i] != i
        if sum(mask) > k:
            mask = mask[:k]

        ret_inds[i] = inds[i][mask]
        ret_dists[i] = dists[i][mask]
    return ret_dists, ret_inds

def brute_closest_points(points, k=1):
    n = len(points)
    dists = pdist(points) #n-1, n-2, ..., n-3
    inds = np.argsort(dists)[:k]


    ind_arr = np.zeros((n*(n-1)/2, 2), dtype=int)
    row = 0

    for i in xrange(n):
        for j in xrange(i+1,n):
            ind_arr[row] = np.array([i,j])
            row += 1


    return ind_arr[inds], dists[inds]

def remove_duplicate_points(points, dists):
    points = np.sort(points)
    max_ind = np.max(points, axis=None)
    keys = max_ind * points[:,0] +  points[:,1]
    keys, inds = np.unique(keys, return_index=True)
    return points[inds], dists[inds]

def metric(x,y):
    return np.sqrt(np.sum((x-y)**2))

def closest_points(points, k=1, tol=1e-11):
    """
    Given an array of points in Euclidean space, find the k pairs of points which are closest.
    """
    n = points.shape[0]
  #  print n, points.shape
    if not n: return np.empty((0,2), dtype=int), np.array([])

    l = int(ceil((4*k)/float(n)))
    if n*n <= 4*k or n <= l:
        return brute_closest_points(points,k)

    dists, inds = all_closest(points,l)
    npairs =  2*k

    #find the indices corresponding to the closest 2*k distances, allowing repetition
    #These aren't necessarily sorted, except for the very last distance
    #Returns a tuple of row numbers (source points and column numbers (number of neighbor) 
    best_k_inds = np.unravel_index(np.argpartition(dists, npairs, axis=None)[:npairs], dists.shape)

    #Use these indices to get the corresponding pairs of points
    pairs = np.empty((npairs, 2), dtype=int)
    pairs[:,0] = best_k_inds[0]
    pairs[:,1] = inds[best_k_inds]

    neighbor_counter = Counter(pairs[:,0])
    
    #the distances for these pairs
    pdists = dists[best_k_inds]
    pairs, pdists = remove_duplicate_points(pairs, pdists)
    
    #filter only the points that have all l nearest neighbors closer than the furthest pair so far
    #we then recursively apply the algorithm to this set of points
    #Figure out how many times each source index is repeated
    mask = np.array([source for source in neighbor_counter if neighbor_counter[source] >= l], dtype=int)

    S = points[mask]
    s_closest, s_dists = closest_points(S, k, tol)
    index_list = mask

    firsts = index_list[s_closest[:,0]]
    seconds = index_list[s_closest[:,1]]
    all_points = np.zeros((len(firsts)+len(pdists),2), dtype = int)
    all_points[:len(pdists)] = pairs
    all_points[len(pdists):] = np.vstack([firsts, seconds]).T

    all_dists = np.concatenate([pdists, s_dists])
    all_points, all_dists = remove_duplicate_points(all_points, all_dists)
    final_best_k = np.argsort(all_dists)[:k]
    return all_points[final_best_k], all_dists[final_best_k]

In [4]:
def shortest_path(g, A):
    """
    predicts links using shortest path method
    """
    lengths = nx.shortest_path_length(g)
    nodes = g.nodes()
    num_nodes = len(nodes)

    scores = np.zeros((num_nodes, num_nodes))
    rev_dict = {nodes[i]: i for i in range(num_nodes)}
    for n1 in lengths:
            for n2 in lengths[n1]:
                    scores[rev_dict[n1]][rev_dict[n2]] = lengths[n1][n2]
    return -1*scores

def common_neighbors(g, A):
    """
    """
    scores = np.dot(A,A)
    return scores

def preferential_attachment(g, A):
    degrees = np.sum(A, axis=1)
    scores = np.outer(degrees, degrees)
    return scores

def jacard(g, A):
    num = np.dot(A,A)
    # finish
    return

def katz(g, A):
    return

def delete_duplicates(pairs):
    """
    deletes duplicate pairs from list pairs
    """
    n = len(pairs)
    copy = []
    for pair in pairs:
        copy.append(sorted(pair))

    for i in range(1,n):
        if copy.count(copy[n-i]) > 1:
            del copy[n-i]
    return copy

def get_loc(pos, n):
    """
    takes position in flattened array and gives location in non-flattened
    """
    row = int(pos/n)
    col = pos % n
    return (row, col)

def complement(first, second):
    """
    returns the compliment of the first list in the second
    """
    second = set(second)
    return [item for item in first if item not in second]


def backward_dict(pairs, nodes):
    """
    """
    n = len(pairs)
    actual = [(nodes[pair[0]],nodes[pair[1]]) for pair in pairs]
    return actual

def predict(scores, nodes, A, k):
    """
    """
    n = len(nodes)
    print('here')
    pred = np.asarray(np.argsort(-1*(scores-10*A-10*np.eye(n)).reshape(n*n)))[0]
    print(pred)
    prediction = []
    for p in pred[:2*k]:
        prediction.append(get_loc(p,n))
    prediction = delete_duplicates(prediction)[:k]
    return backward_dict(prediction, nodes)

def accuracy(prediction, to_predict):
    return len(set(prediction)&set(to_predict))/len(to_predict)

def partition(M, r, c):
    """
    returns the r rows and c columns of matrix M
    """
    part = []
    for x in r:
        for y in c:
            part.append(M[x,y])
    return np.array(part).reshape((len(r),len(c)))

def scipy_eigsh(M, dim=1, tol=1e-8):
    """
    returns the eigenvalue of largest magnitude corresponding to matrix M
    """
    M = M.astype(np.float64)
    sigma = sla.eigsh(M, k=dim, which='LM', tol=tol, return_eigenvectors=False)
    return sigma[0]

def brute_effective_transition(M,nodes,k):
    """
    predicts k links for network with adjacency matrix M
    """
    n = M.shape[0]
    r = list(range(n))
    eigval = scipy_eigsh(M)

    R = np.zeros((n,n))
    for i in range(n):
        print(i)
        for j in range(n):
            if i != j:
                first = csc_matrix(partition(M,[i,j],[i,j]))
                comp = complement(r,[i,j])
                second = csc_matrix(partition(M,[i,j],comp))
                stuff = csc_matrix(partition(M, comp, comp)-eigval*np.eye(n-2))
                third = sla.inv(stuff)
                fourth = csc_matrix(partition(M, comp,[i,j]))
                stuff = second.dot(third.dot(fourth))
                R[i,j] = (first - stuff)[0,1]

    pred = np.asarray(np.argsort(-1*(R - 10*M).reshape(n*n)))[0]

    prediction = []
    for p in pred[:2*k]:
        prediction.append(get_loc(p,n))

    almost = delete_duplicates(prediction)[:k]
    return backward_dict(almost, nodes)

def weighted_effective_transition(A,k):
    """
    predicts k links for network with adjacency matrix M
    """
    n = A.shape[0]
    degrees = np.sum(A, axis=1)
    M = A/d
    r = list(range(n))
    eigval = scipy_eigsh(M)

    R = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i != j:
                first = partition(M,[i,j],[i,j])
                comp = complement(r,[i,j])
                second = partition(M,[i,j],comp)
                temp = partition(M, comp, comp)
                third = np.linalg.inv(partition(M, comp, comp)-eigval*np.identity(n-2))
                fourth = partition(M, comp,[i,j])
                R[i,j] = (first - np.dot(second,np.dot(third,fourth)))[0,1]

    pred = np.asarray(np.argsort(-1*(R - 10*M).reshape(n*n)))[0]

    prediction = []
    for p in pred[:2*k]:
        prediction.append(get_loc(p,n))

    almost = delete_duplicates(prediction)[:k]
    return backward_dict(almost, nodes)


In [7]:
shortest_path(G,A)

array([[-0., -1., -1., ..., -2., -2., -2.],
       [-1., -0., -1., ..., -2., -2., -1.],
       [-1., -1., -0., ..., -2., -2., -2.],
       ...,
       [-2., -2., -2., ..., -0., -3., -3.],
       [-2., -2., -2., ..., -3., -0., -3.],
       [-2., -1., -2., ..., -3., -3., -0.]])

In [18]:
methods = []
times = []
accs = []

In [6]:
filename = '/home/byu.local/brynbb/myacmeshare/link_prediction/data/haggledata.txt'
G, to_predict = load_data(filename)
nodes = G.nodes()
A = nx.adjacency_matrix(G)

In [8]:
print(len(to_predict))

319


In [9]:
A.shape

(178, 178)

In [19]:
k = 5

start = time.time()
scores = shortest_path(G,A)
print(scores)
prediction = predict(scores, nodes, A, k)
print(prediction)
acc = accuracy(prediction, to_predict)
methods.append('Shortest Path')
times.append(time.time()-start)
accs.append(acc)
print('done with shortest')

"""start = time.time()
prediction = brute_effective_transition(A,nodes, k)
acc = accuracy(prediction, to_predict)
methods.append('Effective Transition')
times.append(time.time()-start)
accs.append(acc)
print('done with shortest')

start = time.time()
prediction = brute_effective_transition(A, nodes,k)
acc = accuracy(prediction, to_predict)
methods.append('Weighted Effective Transition')
times.append(time.time()-start)
accs.append(acc)
print('done with shortest')"""

start = time.time()
scores = common_neighbors(G, A)
prediction = predict(scores, nodes, A, k)
acc = accuracy(prediction, to_predict)
methods.append('Common Neighbors')
times.append(time.time()-start)
accs.append(acc)
print('done with common neighbors')

start = time.time()
prediction = spectral_embed(G, k, dim=4)
acc = accuracy(prediction, to_predict)
methods.append('Spectral Embedding')
times.append(time.time()-start)
accs.append(acc)
print('done with spectral embed')

[[-0. -1. -1. ... -2. -2. -2.]
 [-1. -0. -1. ... -2. -2. -1.]
 [-1. -1. -0. ... -2. -2. -2.]
 ...
 [-2. -2. -2. ... -0. -3. -3.]
 [-2. -2. -2. ... -3. -0. -3.]
 [-2. -1. -2. ... -3. -3. -0.]]
here
[28337 12680 12681 ...  7028  7130  5176]
[(36, 235), (43, 83), (44, 83), (45, 83), (198, 200)]
done with shortest
here
[ 2327  2506  9666 ...  9777 29958   884]
done with common neighbors


NameError: name 'spectral_embed' is not defined

In [20]:
times

[0.03240466117858887, 0.0032432079315185547]

In [21]:
accs

[0.0, 0.0]